In [1]:
import numpy as np
import pandas as pd
from tkinter import *
import tkinter.font as tkfont
import math

In [2]:
df_games=pd.read_csv('games_details_dataset.csv')

In [3]:
def sortByRate(game_id):
    #binary search for faster results
    start_search = 0
    numOfValues = df_games.shape[0]
    for k in range(10):
        if game_id == df_games.iloc[start_search + math.floor(numOfValues/2)]['appid']:
            return df_games.iloc[start_search + math.floor(numOfValues/2)]['rating']
        elif game_id > df_games.iloc[start_search + math.floor(numOfValues/2)]['appid']:
            start_search = start_search + math.floor(numOfValues/2) + 1
            numOfValues = math.ceil(numOfValues/2)
        else:
            numOfValues = math.ceil(numOfValues/2)       
    for i in range(start_search, start_search + numOfValues):
        if game_id == df_games.iloc[i]['appid']:
            return df_games.iloc[i]['rating']

In [4]:
def search_process(new_input):
    matched_games = []
    for i in range(df_games.shape[0]):
        #title match
        if new_input[5] in df_games.iloc[i]['name'] or new_input[5] == "":
            #genre match
            if new_input[0] in df_games.iloc[i]['genres'] or new_input[0]=="":
                #developer match
                if new_input[3] in df_games.iloc[i]['developer'] or new_input[3]=="":
                    #publisher match
                    if new_input[2] in df_games.iloc[i]['publisher'] or new_input[2]=="":
                        #category match
                        if new_input[1] in df_games.iloc[i]['categories'] or new_input[1]=="":
                            #platform match
                            if df_games.iloc[i][new_input[4]]==1:
                                matched_games.append(df_games.iloc[i]['appid'])
    matched_games.sort(key=sortByRate, reverse=True)
    return matched_games

In [5]:
def recommend():
    print("Nothing yet")

In [6]:
def Search():
    
    def clicked():
        new_input=[genre.get(),category.get(),publisher.get(),developer.get(),platform_choice.get(),title.get()]
        ans=search_process(new_input)
        for i in range(df_games.shape[0]):
            if df_games.iloc[i]["appid"] in ans:
                print(df_games.iloc[i][:])
        
    window_search = Toplevel(window)
    window_search.title("Search for games")
    window_search.geometry('700x500')
    window_search.configure(bg= '#6696ad')
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    title_label = Label(window_search, font=('arial',9,'bold'),text='Title',bg = "#6696ad")
    title_label.pack()
    title = Entry(window_search,width=20)
    title.var = StringVar()
    title['textvariable'] = title.var
    #title.var.trace_add('write', toggle_state)
    title.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    class AutocompleteEntry(Entry):
        def __init__(self, autocompleteList, *args, **kwargs):
            # Listbox length
            
            if 'listboxLength' in kwargs:
                self.listboxLength = kwargs['listboxLength']
                del kwargs['listboxLength']
            else:
                self.listboxLength = 6

            # Custom matches function
            if 'matchesFunction' in kwargs:
                self.matchesFunction = kwargs['matchesFunction']
                del kwargs['matchesFunction']
            else:
                def matches(fieldValue, acListEntry):
                    pattern = re.compile('.*' + re.escape(fieldValue) + '.*', re.IGNORECASE)
                    return re.match(pattern, acListEntry)

                self.matchesFunction = matches


            Entry.__init__(self,*args, **kwargs)
            self.focus()

            self.autocompleteList = autocompleteList

            self.var = self["textvariable"]
            if self.var == '':
                self.var = self["textvariable"] = StringVar()

            self.var.trace('w', self.changed)
            self.bind("<Return>", self.selection)
            self.bind("<Up>", self.moveUp)
            self.bind("<Down>", self.moveDown)

            self.listboxUp = False

        def changed(self, name, index, mode):
            if self.var.get() == '':
                if self.listboxUp:
                    self.listbox.destroy()
                    self.listboxUp = False
            else:
                words = self.comparison()
                if words:
                    if not self.listboxUp:
                        self.listbox = Listbox(window_search, width=self["width"], height=self.listboxLength)
                        self.listbox.bind("<Button-1>", self.selection)
                        self.listbox.bind("<Double-Button-1>", self.selection)
                        self.listbox.bind("<Return>", self.selection)
                        self.listbox.place(x=self.winfo_x(), y=self.winfo_y() + 20)
                        self.listboxUp = True

                    self.listbox.delete(0, END)
                    for w in words:
                        self.listbox.insert(END,w)
                else:
                    if self.listboxUp:
                        self.listbox.destroy()
                        self.listboxUp = False

        def selection(self, event):
            if self.listboxUp:
                self.var.set(self.listbox.get(ACTIVE))
                self.listbox.destroy()
                self.listboxUp = False
                self.icursor(END)

        def moveUp(self, event):
            if self.listboxUp:
                if self.listbox.curselection() == ():
                    index = '0'
                else:
                    index = self.listbox.curselection()[0]

                if index != '0':                
                    self.listbox.selection_clear(first=index)
                    index = str(int(index) - 1)

                    self.listbox.see(index) # Scroll!
                    self.listbox.selection_set(first=index)
                    self.listbox.activate(index)

        def moveDown(self, event):
            if self.listboxUp:
                if self.listbox.curselection() == ():
                    index = '0'
                else:
                    index = self.listbox.curselection()[0]

                if index != END:                        
                    self.listbox.selection_clear(first=index)
                    index = str(int(index) + 1)

                    self.listbox.see(index) # Scroll!
                    self.listbox.selection_set(first=index)
                    self.listbox.activate(index) 

        def comparison(self):
            return [ w for w in self.autocompleteList if self.matchesFunction(self.var.get(), w) ]
     
    genre_file = open("genre.txt", "r")
    autocompleteList_genre = genre_file.readlines()
    autocompleteList_genre = [x.replace("\n","") for x in autocompleteList_genre]
    
    publisher_file = open("publisher.txt", "r")
    autocompleteList_publisher = publisher_file.readlines()
    autocompleteList_publisher = [x.replace("\n","") for x in autocompleteList_publisher]
    
    developer_file = open("developer.txt", "r")
    autocompleteList_developer = developer_file.readlines()
    autocompleteList_developer = [x.replace("\n","") for x in autocompleteList_developer]
    
    category_file = open("category.txt", "r")
    autocompleteList_category = category_file.readlines()
    autocompleteList_category = [x.replace("\n","") for x in autocompleteList_category]
    
    platformOptions = ["windows", "linux", "mac"]
    
    def matches(fieldValue, acListEntry):
        pattern = re.compile(re.escape(fieldValue) + '.*', re.IGNORECASE)
        return re.match(pattern, acListEntry)
    
    genre_label=Label(window_search, font=('arial',9,'bold'),text='Genre',bg = "#6696ad")
    genre_label.pack()
    genre = AutocompleteEntry(autocompleteList_genre, window_search, listboxLength=10, matchesFunction=matches)
    genre.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    publisher_label=Label(window_search, font=('arial',9,'bold'),text='Publisher', bg = "#6696ad")
    publisher_label.pack()
    publisher = AutocompleteEntry(autocompleteList_publisher, window_search, listboxLength=10, matchesFunction=matches)
    publisher.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    developer_label=Label(window_search, font=('arial',9,'bold'),text='Developer', bg = "#6696ad")
    developer_label.pack()
    developer = AutocompleteEntry(autocompleteList_developer, window_search, listboxLength=10, matchesFunction=matches)
    developer.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    category_label=Label(window_search, font=('arial',9,'bold'),text='Category', bg = "#6696ad")
    category_label.pack()
    category = AutocompleteEntry(autocompleteList_category, window_search, listboxLength=10, matchesFunction=matches)
    category.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    
    platform_label=Label(window_search, font=('arial',9,'bold'),text='Platform', bg = "#6696ad")
    platform_label.pack()
    platform_choice = StringVar()
    platform_choice.set("windows")
    platform = OptionMenu( window_search , platform_choice , *platformOptions )
    platform.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()
    btn = Button(window_search, relief=SOLID, state = 'normal', text = "Result" , activebackground='#6696ad',activeforeground='white', bg='#d68ba0', fg='White', command = clicked, font=button_font)
    btn.pack()
    
    space = Label(window_search, text="", bg = "#6696ad").pack()

    window_search.mainloop()

In [7]:
window = Tk()
window.title("Games Recommender System")
window.geometry('700x394')

bg = PhotoImage(file = "app_background.png")

canvas1 = Canvas( window, width = 1000, height = 1000)
canvas1.pack(fill = "both", expand = True)
canvas1.create_image( 0, 0, image = bg, anchor = "nw")


button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
btn_search = Button(window,text='Search for games',relief=SOLID,
                    activebackground='#667fa1',activeforeground='white', bg='#d68ba0', fg='White',
                    command=Search, font=button_font)

btn_search.pack()

btn_recommend = Button(window,text='Game recommendations',relief=SOLID,
                    activebackground='#667fa1',activeforeground='white', bg='#d68ba0', fg='White',
                    command=recommend, font=button_font)

btn_recommend.pack()


button1_canvas = canvas1.create_window(95, 170, anchor = "nw", window = btn_search)

button2_canvas = canvas1.create_window(400, 170, anchor = "nw", window = btn_recommend)

    
window.mainloop()

appid                                                           22380
name                                                Fallout New Vegas
release_date                                                 21-10-10
english                                                             1
developer                                      Obsidian Entertainment
publisher                                          Bethesda Softworks
required_age                                                       18
categories          Single-player;Steam Achievements;Partial Contr...
genres                                                     Action;RPG
steamspy_tags                         Open World;RPG;Post-apocalyptic
achievements                                                       75
positive_ratings                                                66756
negative_ratings                                                 3149
average_playtime                                                 2485
median_playtime     

appid                                                          303430
name                                                       Roundabout
release_date                                                 18-09-14
english                                                             1
developer                                                   No Goblin
publisher                                                   No Goblin
required_age                                                        0
categories          Single-player;Steam Achievements;Full controll...
genres                                  Action;Adventure;Casual;Indie
steamspy_tags                                       FMV;Indie;Driving
achievements                                                       40
positive_ratings                                                  425
negative_ratings                                                   31
average_playtime                                                  229
median_playtime     

appid                                                          384940
name                                                 Magical Brickout
release_date                                                 17-10-16
english                                                             1
developer                                         Cunning Force Games
publisher                                         Cunning Force Games
required_age                                                        0
categories          Single-player;Steam Achievements;Full controll...
genres                                            Action;Casual;Indie
steamspy_tags                                     Action;Indie;Casual
achievements                                                       44
positive_ratings                                                   19
negative_ratings                                                    4
average_playtime                                                    0
median_playtime     

appid                                                          458110
name                                                         VReakout
release_date                                                 13-07-16
english                                                             1
developer                                             Animate Objects
publisher                                             Animate Objects
required_age                                                        0
categories          Single-player;Steam Achievements;Steam Leaderb...
genres                                            Action;Casual;Indie
steamspy_tags                                     Action;Casual;Indie
achievements                                                       24
positive_ratings                                                    6
negative_ratings                                                    1
average_playtime                                                    0
median_playtime     

appid                                                          599080
name                                                The Blackout Club
release_date                                                 30-10-18
english                                                             1
developer                                                    Question
publisher                                                    Question
required_age                                                       18
categories          Single-player;Multi-player;Online Multi-Player...
genres                           Action;Indie;Simulation;Early Access
steamspy_tags                         Early Access;Horror;Multiplayer
achievements                                                       14
positive_ratings                                                  461
negative_ratings                                                   58
average_playtime                                                    0
median_playtime     

appid                                                 798950
name                                                  Logout
release_date                                        01-05-18
english                                                    1
developer                                  PlayWrite Studios
publisher                                 PlayWrite Studios 
required_age                                               0
categories          Single-player;Partial Controller Support
genres                   Action;Adventure;Free to Play;Indie
steamspy_tags                  Free to Play;Adventure;Action
achievements                                               0
positive_ratings                                          22
negative_ratings                                          13
average_playtime                                           0
median_playtime                                            0
owners                                               0-20000
price                   